<a href="https://colab.research.google.com/github/Zelechos/Generador-Css/blob/main/GeneradorCSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Generador de CSS [v0.0.8]**

# Clonamos nuestro repositorio en donde se encuentra nuestro dataset con este comando

In [2]:
!git clone https://github.com/Zelechos/Generador-Css.git

Cloning into 'Generador-Css'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 58 (delta 21), reused 13 (delta 3), pack-reused 0
Unpacking objects: 100% (58/58), done.


### en este caso no es necesario ya que tenemos el dataset en github por ende tenemos que trabajar con el dataset que esta en formato json

### Accedemos a los datos del dataset TEST.json


In [3]:
import json

with open('/content/Generador-Css/dataset/TEST.json') as file:
    data = json.load(file)

#accedemos a la los datos
data_styles = data["styles"]
print(data_styles[0])


#accediendo a los estilos
styles = data_styles[0]["style"]
print("\nstyles access => ",styles)


#accediendo a los selectores
selector = data_styles[0]["selector"]
print("\nselector access => ",selector)


{'style': 'body{overflow:hidden;background:rgb(25,35,125)}', 'selector': 'body'}

styles access =>  body{overflow:hidden;background:rgb(25,35,125)}

selector access =>  body


### Creamos un array para poder mandar los datos y luego tranformarlos en numeros

In [4]:
data_css = []
styles = []
selectors = []
for data in data_styles:
  selectors.append(data["selector"])
  styles.append(data["style"])
 
# print(selectors)
# print(styles)
data_css.append(selectors)
data_css.append(styles)
print(data_css[0])
print(data_css[1])


['body', 'div.drop-container', 'div.drop', 'h1', '@font-face', 'body', '.logo', '.switch-left', '.pad-left', '.switch-right', '.pad-right', '.line', '.text', '.text', '.text', '.reload', '.logo', 'html,body', 'body', '*', '*', '*', '.css_lab', '.witch', 'body', 'body img', 'img', 'body h1', 'h1', 'body a', 'a', '@keyframes glow', 'from', 'to', 'body', 'h1', 'ul', 'li', 'li:hover span', 'li.invisible', 'li.animate', 'span', 'span:before', 'html', 'body', 'header', 'article', 'h3,section', 'h3', 'section', 'h3:before', 'h3:before,section:before', 'section:before', 'h3', 'section', 'h3:before', 'section:before', 'p', 'a', 'body,html', 'body', 'html', 'div.codepen', 'div', '.codepen', '*', 'body', 'aside.context', '.context', 'aside', 'a', 'a:hover', 'footer', 'footer a', 'a']
['body{overflow:hidden;background:rgb(25,35,125)}', 'div.drop-container{position:absolute;top:0;right:0;bottom:0;left:0;margin:auto;height:200px;width:200px}', 'div.drop{position:absolute;top:-25%;width:100%;height:1

### Convertimos los datos a numeros para trabajar

### Pasos a Seguir para construir mi Custom Dataset
1.   Contruir el vocabulario para asignar a cada palabra
2.   Configurar un dataset de Pytorch para cargar nuestro datos
2.   Configurar el batch para que todos los ejemplos tengan las misma longitud (seq_len) y tambien configuramos el dataloader




## Importaciones requeridas

In [5]:
import torch
import torch.nn as nn
import spacy # para la tokenizacion
import pandas as pd # para buscar en el archivo de anotaciones (atento)
from torch.nn.utils.rnn import pad_sequence #para el lote de almohadillas (pad batch) para que todos los datos tengan la misma longitud
from torch.utils.data import DataLoader , Dataset #para crea el dataset y el dataloader para el entrenamiento


device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Clase para crear nuestro vocabulario

In [6]:
spacy_eng = spacy.load("en")
class Vocabulary:

  def __init__(self, freq_threshold):
    self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}
    self.stoi = {"<PAD>":0, "<SOS>":1, "<EOS>":2, "<UNK>":3}
    self.freq_threshold = freq_threshold

  def __len__(self):
    #devolvemos la longitud de nuestro vocabulario
    return len(self.itos)

  #metodo para definir el ingles de nuestro vocabulario
  @staticmethod
  def tokenizer_eng(text):
    return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

  #ESTO ES LO QUE HACER EJEMPLO!!!
  # "body{background-color:red;}"" => ["body","{","background","-","color",":","red",";","}"]

  #construimos el vocabulario!!
  def build_vocabulary(self, sentence_list):
    frequencies = {}
    idx = 4

    for sentence in sentence_list:
      for word in self.tokenizer_eng(sentence):
        if word not in frequencies:
          frequencies[word] = 1
        else:
          frequencies[word] += 1

        if frequencies[word] == self.freq_threshold:
          self.stoi[word] = idx
          self.itos[idx] = word
          idx += 1

  def numericalize(self, text):
    tokenized_text = self.tokenizer_eng(text)
    return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
    ]

### Clase CssDataset para clasificar el conjunto de datos nuestro dataset

In [36]:
class CssDataset(Dataset):

  def __init__ (self, css_styles, freq_threshold = 5):
    self.df = css_styles

    # vamos a obtener nuestros selectores
    self.selectors = self.df[0]

    # vamos a obtener nuestros styles
    self.styles = self.df[1]
    
    #Incializamos el Vocabulario (IMPORTANTEE!!!)
    self.vocab = Vocabulary(freq_threshold)
    self.vocab1 = Vocabulary(freq_threshold)

    self.vocab.build_vocabulary(self.styles) #Pasamos las Listas a el vocabulario
    self.vocab1.build_vocabulary(self.selectors) #Pasamos las Listas a el vocabulario
    

  def __len__(self):
    return len(self.df[1]) #devolvemos la longitud de nuestros dataset

  def __getitem__(self, index):
    selector = self.selectors[index]
    style = self.styles[index]

    numericalized_style = [self.vocab.stoi["<SOS>"]] 
    numericalized_style += self.vocab.numericalize(style) 
    numericalized_style.append(self.vocab.stoi["<EOS>"]) 

    numericalized_selector = [self.vocab1.stoi["<SOS>"]] 
    numericalized_selector += self.vocab1.numericalize(selector) 
    numericalized_selector.append(self.vocab1.stoi["<EOS>"]) 

    return torch.tensor(numericalized_selector) , torch.tensor(numericalized_style) 


train = CssDataset(data_css)
len(train)
print(type(train))

<class '__main__.CssDataset'>


## Clase para cargar los datos

In [51]:
class MyCollate:

  def __init__(self, pad_idx):
    self.pad_idx = pad_idx

  def __call__(self, batch):
    targets = [item[0] for item in batch]
    targets = pad_sequence(targets, batch_first = False, padding_value = self.pad_idx )

    targets1 = [item[1] for item in batch]
    targets1 = pad_sequence(targets1, batch_first = False, padding_value = self.pad_idx )

    return targets, targets1

def get_loader(css_styles, batch_size = 32, num_workers = 8, shuffle = True, pin_memory = True):
  dataset = CssDataset(css_styles)
  pad_idx = dataset.vocab.stoi["<PAD>"]

  dataloader = DataLoader(
      dataset = dataset,
      batch_size = batch_size,
      num_workers = num_workers,
      shuffle= shuffle,
      pin_memory=pin_memory,
      collate_fn=MyCollate(pad_idx=pad_idx),
  )

  return dataset, dataloader

def main():
  dataset, dataloader = get_loader(data_css)
  print(dataloader)
  print(dataset)
  # for idx , (selectors, styles) in enumerate(dataloader):
    # print(selectors.shape)
    # print(styles.shape)
    # print(selectors)
    # print(styles)
    # print(type(selectors))
    # print(type(styles))
    # print(idx)

if __name__ == "__main__" :
  main()



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Usaremos Perceptrón Multicapa (MLP) para el Generador y Discriminador

In [52]:
class CharRNN(torch.nn.Module):
  def __init__(self, input_size, embedding_size=128, hidden_size=256, num_layers=2, dropout=0.2):
    super().__init__()
    self.encoder = torch.nn.Embedding(input_size, embedding_size)
    self.rnn = torch.nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, input_size)

  def forward(self, x):
    x = self.encoder(x)
    x, h = self.rnn(x)         
    y = self.fc(x[:,-1,:])
    return y

In [53]:
model = CharRNN(input_size=114)
outputs = model(torch.randint(0, 114, (64, 50)))
outputs.shape

torch.Size([64, 114])

In [54]:
from tqdm import tqdm
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

def fit(model, dataloader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader)
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            bar.set_description(f"loss {np.mean(train_loss):.5f}")
        bar = tqdm(dataloader)
        val_loss = []
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                bar.set_description(f"val_loss {np.mean(val_loss):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f}")

def predict(model, X):
    model.eval() 
    with torch.no_grad():
        X = torch.tensor(X).to(device)
        pred = model(X.unsqueeze(0))
        return pred

In [55]:
dataset, dataloader = get_loader(data_css)
print(len(dataloader))
print(type(dataloader))
i , ss , sss = dataloader

# print(sss)
model = CharRNN(input_size=114)
fit(model, dataloader, epochs=20)

3
<class 'torch.utils.data.dataloader.DataLoader'>


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/3 [00:00<?, ?it/s]


ValueError: ignored